In [1]:
import pandas as pd

In [ ]:
df_accepted = pd.read_csv("data/1-rmv_dup_introns_gencode_v44.tsv", sep="\t")
df_accepted["class"] = 1
df_accepted

In [ ]:
df_rejected = pd.read_csv("data/1.2_original_rejected_introns", sep="\t")
df_rejected = df_rejected.drop(columns="Duplicate_Count")
df_rejected["class"] = 0
df_rejected

In [ ]:
df_merged = pd.concat([df_accepted, df_rejected])
df_merged

# Simulating false introns - Workflow

<pre>
The set of simulated false introns is generated using:
1. Simulate false read using pbsim3
a*. ONT single pass - protein coding genes `pbsim --seed 1 --strategy templ --method qshmm --qshmm ./data/pbsim_models/QSHMM-ONT.model --template ./data/gencode/gencode.v44.pc_transcripts.fa --prefix pbsim_rna_a --difference-ratio 39:24:36`
b*. ONT multi-pass - protein coding genes `pbsim --seed 2 --strategy templ --method qshmm --qshmm ./data/pbsim_models/QSHMM-ONT.model --template ./data/gencode/gencode.v44.pc_transcripts.fa --prefix pbsim_rna_b --difference-ratio 39:24:36 --pass-num 3`
c*. PacBio - Iso-seq - protein coding genes `pbsim --seed 3 --strategy templ --method qshmm --qshmm ./data/pbsim_models/QSHMM-RSII.model --template ./data/gencode/gencode.v44.pc_transcripts.fa --prefix pbsim_rna_c --difference-ratio 22:45:33 --accuracy-mean 0.99 --pass-num 3`
d*. ONT single pass - lcnRNA `pbsim --seed 4 --strategy templ --method qshmm --qshmm ./data/pbsim_models/QSHMM-ONT.model --template ./data/gencode/gencode.v44.lncRNA_transcripts.fa --prefix pbsim_rna_d --difference-ratio 39:24:36`

2. Read-alignment using Minimap2
a*. `minimap2 -ax splice -t 6 -u f --seed 1 -k 14 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa pbsim_rna_a.fastq > aln_a.sam`
b*. `samtools fastq ./pbsim_rna_b.sam > simulated_b.fq`  then
    `minimap2 -ax splice -t 6 -u f --seed 2 -k 14 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa ./simulated_b.fq > aln_b.sam`
c*. `samtools fastq ./pbsim_rna_c.sam > simulated_c.fq` then
    `minimap2 -ax splice:hq -u f --seed 3 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa ./simulated_c.fq > aln_c.sam`
d*. `minimap2 -ax splice -t 6 -u f --seed 4 -k 14 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa ./pbsim_rna_d > aln_d.sam`

Repeat for * equals to [a,b,c,d]:
    3. Sorted and convert to BAM
    `samtools sort ./aln_*.sam > aln_*_sorted.bam`

    4. Indexing
    `samtools index aln_*_sorted.bam`



## Simulation a

<pre>
:::: Simulation parameters :::

strategy : templ
method : qshmm
qshmm : ./data/pbsim_models/QSHMM-ONT.model
template : ./data/gencode/gencode.v44.pc_transcripts.fa
prefix : pbsim_rna_a
id-prefix : S
difference-ratio : 39:24:36
seed : 1
accuracy-mean : 0.850000
pass_num : 1
hp-del-bias : 1.000000

:::: Template stats ::::

file name : ./data/gencode/gencode.v44.pc_transcripts.fa
template num. : 110962
template total length : 265796693

:::: Simulation stats ::::

read num. : 110962
read length mean (SD) : 2353.105090 (2313.062189)
read length min : 8
read length max : 107547
read accuracy mean (SD) : 0.849757 (0.045486)
substitution rate. : 0.058598
insertion rate. : 0.036047
deletion rate. : 0.054015

:::: System utilization ::::

CPU time(s) : 9
Elapsed time(s) : 10
</pre>

<pre>
[M::mm_idx_gen::30.213*1.67] collected minimizers
[M::mm_idx_gen::37.292*2.48] sorted minimizers
[M::main::37.292*2.48] loaded/built the index for 194 target sequence(s)
[M::mm_mapopt_update::38.088*2.45] mid_occ = 2053
[M::mm_idx_stat] kmer size: 14; skip: 5; is_hpc: 0; #seq: 194
[M::mm_idx_stat::38.533*2.44] distinct minimizers: 65621209 (23.06% are singletons); average occurrences: 15.289; average spacing: 3.090; total length: 3099750718
[M::worker_pipeline::171.215*5.14] mapped 110962 sequences
[M::main] Version: 2.26-r1175
[M::main] CMD: minimap2 -ax splice -t 6 -u f --seed 1 -k 14 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa pbsim_rna_a.fastq
[M::main] Real time: 171.384 sec; CPU: 880.425 sec; Peak RSS: 18.422 GB`
</pre>

<pre>
Command:
samtools sort ./aln_a.sam > aln_a_sorted.bam

Output:
</pre>

<pre>
Command:
samtools index aln_a_sorted.bam

Output:
</pre>

## Simulation b

<pre>
Command:
pbsim --seed 2 --strategy templ --method qshmm --qshmm ./data/pbsim_models/QSHMM-ONT.model --template ./data/gencode/gencode.v44.pc_transcripts.fa --prefix pbsim_rna_b --difference-ratio 39:24:36 --pass-num 3

Output:
:::: Simulation parameters :::

strategy : templ
method : qshmm
qshmm : ./data/pbsim_models/QSHMM-ONT.model
template : ./data/gencode/gencode.v44.pc_transcripts.fa
prefix : pbsim_rna_b
id-prefix : S
difference-ratio : 39:24:36
seed : 2
accuracy-mean : 0.850000
pass_num : 3
hp-del-bias : 1.000000

:::: Template stats ::::

file name : ./data/gencode/gencode.v44.pc_transcripts.fa
template num. : 110962
template total length : 265796693

:::: Simulation stats ::::

read num. : 110962
read length mean (SD) : 2353.024185 (2313.031686)
read length min : 7
read length max : 107697
read accuracy mean (SD) : 0.849485 (0.045554)
substitution rate. : 0.058784
insertion rate. : 0.036170
deletion rate. : 0.054173

:::: System utilization ::::

CPU time(s) : 36
Elapsed time(s) : 40
</pre>

<pre>
Command:
samtools fastq ./pbsim_rna_b.sam > simulated_b.fq

Output:
[M::bam2fq_mainloop] processed 332886 reads
</pre>

<pre>
Command:
minimap2 -ax splice -t 6 -u f --seed 2 -k 14 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa ./simulated_b.fq > aln_b.sam

Output:
[M::mm_idx_gen::30.944*1.65] collected minimizers
[M::mm_idx_gen::38.007*2.45] sorted minimizers
[M::main::38.007*2.45] loaded/built the index for 194 target sequence(s)
[M::mm_mapopt_update::38.795*2.42] mid_occ = 2053
[M::mm_idx_stat] kmer size: 14; skip: 5; is_hpc: 0; #seq: 194
[M::mm_idx_stat::39.234*2.40] distinct minimizers: 65621209 (23.06% are singletons); average occurrences: 15.289; average spacing: 3.090; total length: 3099750718
[M::worker_pipeline::308.814*5.53] mapped 202494 sequences
[M::worker_pipeline::432.635*5.65] mapped 130392 sequences
[M::main] Version: 2.26-r1175
[M::main] CMD: minimap2 -ax splice -t 6 -u f --seed 2 -k 14 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa ./simulated_b.fq
[M::main] Real time: 432.756 sec; CPU: 2442.688 sec; Peak RSS: 18.422 GB
</pre>


<pre>
Command:
samtools sort ./aln_b.sam > aln_b_sorted.bam

Output:
[bam_sort_core] merging from 4 files...
</pre>


<pre>
Command:
samtools index aln_b_sorted.bam

Output:
</pre>

# Simulation c

<pre>
Command:
pbsim --seed 3 --strategy templ --method qshmm --qshmm ./data/pbsim_models/QSHMM-RSII.model --template ./data/gencode/gencode.v44.pc_transcripts.fa --prefix pbsim_rna_c --difference-ratio 22:45:33 --accuracy-mean 0.999 --pass-num 3

Output:
:::: Simulation parameters :::

strategy : templ
method : qshmm
qshmm : ./data/pbsim_models/QSHMM-RSII.model
template : ./data/gencode/gencode.v44.pc_transcripts.fa
prefix : pbsim_rna_c
id-prefix : S
difference-ratio : 22:45:33
seed : 3
accuracy-mean : 0.990000
pass_num : 3
hp-del-bias : 1.000000

:::: Template stats ::::

file name : ./data/gencode/gencode.v44.pc_transcripts.fa
template num. : 110962
template total length : 265796693

:::: Simulation stats ::::

read num. : 110962
read length mean (SD) : 2406.880397 (2365.994069)
read length min : 7
read length max : 109900
read accuracy mean (SD) : 0.959701 (0.043944)
substitution rate. : 0.008867
insertion rate. : 0.018072
deletion rate. : 0.013296

:::: System utilization ::::

CPU time(s) : 30
Elapsed time(s) : 33
</pre>

<pre>
Command:
samtools fastq ./pbsim_rna_c.sam > simulated_c.fq

Output:
[M::bam2fq_mainloop] processed 332886 reads
</pre>

<pre>
Command:
samtools fastq ./pbsim_rna_c.sam > simulated_c.fq

Output:
[M::bam2fq_mainloop] processed 332886 reads
</pre>

<pre>
Command:
minimap2 -ax splice:hq -u f --seed 3 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa ./simulated_c.fq > aln_c.sam

Output:
[M::mm_idx_gen::30.883*1.67] collected minimizers
[M::mm_idx_gen::42.766*2.04] sorted minimizers
[M::main::42.766*2.04] loaded/built the index for 194 target sequence(s)
[M::mm_mapopt_update::44.277*2.00] mid_occ = 767
[M::mm_idx_stat] kmer size: 15; skip: 5; is_hpc: 0; #seq: 194
[M::mm_idx_stat::45.281*1.98] distinct minimizers: 167225302 (35.42% are singletons); average occurrences: 6.036; average spacing: 3.071; total length: 3099750718
[M::worker_pipeline::432.555*2.89] mapped 197380 sequences
[M::worker_pipeline::624.978*2.92] mapped 135506 sequences
[M::main] Version: 2.26-r1175
[M::main] CMD: minimap2 -ax splice:hq -u f --seed 3 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa ./simulated_c.fq
[M::main] Real time: 625.177 sec; CPU: 1826.355 sec; Peak RSS: 19.006 GB
</pre>

<pre>
Command:
samtools sort ./aln_c.sam > aln_c_sorted.bam

Output:
[bam_sort_core] merging from 3 files...

</pre>

<pre>
Command:
samtools index aln_c_sorted.bam

Output:

</pre>

# Simulation d

<pre>
Command:
pbsim --seed 4 --strategy templ --method qshmm --qshmm ./data/pbsim_models/QSHMM-ONT.model --template ./data/gencode/gencode.v44.lncRNA_transcripts.fa --prefix pbsim_rna_d --difference-ratio 39:24:36

Output:
:::: Simulation parameters :::

strategy : templ
method : qshmm
qshmm : ./data/pbsim_models/QSHMM-ONT.model
template : ./data/gencode/gencode.v44.lncRNA_transcripts.fa
prefix : pbsim_rna_d
id-prefix : S
difference-ratio : 39:24:36
seed : 4
accuracy-mean : 0.850000
pass_num : 1
hp-del-bias : 1.000000

:::: Template stats ::::

file name : ./data/gencode/gencode.v44.lncRNA_transcripts.fa
template num. : 58246
template total length : 77004945

:::: Simulation stats ::::

read num. : 58246
read length mean (SD) : 1298.703568 (2040.613450)
read length min : 32
read length max : 341038
read accuracy mean (SD) : 0.849009 (0.046553)
substitution rate. : 0.058943
insertion rate. : 0.036240
deletion rate. : 0.054227

:::: System utilization ::::

CPU time(s) : 2
Elapsed time(s) : 2
</pre>

<pre>
Command:
minimap2 -ax splice -t 6 -u f --seed 4 -k 14 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa ./pbsim_rna_d.fastq > aln_d.sam

Output:
[M::mm_idx_gen::30.336*1.66] collected minimizers
[M::mm_idx_gen::37.372*2.47] sorted minimizers
[M::main::37.372*2.47] loaded/built the index for 194 target sequence(s)
[M::mm_mapopt_update::38.128*2.44] mid_occ = 2053
[M::mm_idx_stat] kmer size: 14; skip: 5; is_hpc: 0; #seq: 194
[M::mm_idx_stat::38.572*2.42] distinct minimizers: 65621209 (23.06% are singletons); average occurrences: 15.289; average spacing: 3.090; total length: 3099750718
[M::worker_pipeline::91.712*4.12] mapped 58246 sequences
[M::main] Version: 2.26-r1175
[M::main] CMD: minimap2 -ax splice -t 6 -u f --seed 4 -k 14 ./data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa ./pbsim_rna_d.fastq
[M::main] Real time: 91.868 sec; CPU: 377.793 sec; Peak RSS: 18.422 GB
</pre>

<pre>
Command:
samtools sort ./aln_d.sam > aln_d_sorted.bam

Output:

</pre>

<pre>
Command:
samtools index aln_d_sorted.bam

Output:

</pre>

# Extracting introns from the sorted (and indexed) BAM file

In [1]:
import pysam

def get_introns_from_bam(infile, outfile):
    # Modified from a script generated by ChatGPT v3.5

    # Open the SAM file
    samfile = pysam.AlignmentFile(infile, 'r')

    # Open output file for intron coordinates
    output_file = open(outfile, 'w')

    # Iterate over each read in the SAM file
    for read in samfile.fetch():
        # Skip unmapped reads
        if read.is_unmapped:
            continue

        # Parse the CIGAR string to extract intron regions
        cigar_tuples = read.cigartuples
        introns = []
        current_position = read.reference_start
        for cigar in cigar_tuples:
            cigar_type, cigar_length = cigar
            if cigar_type == 3:  # 3 represents the CIGAR operation 'N' for intron
                intron_start = current_position
                intron_end = current_position + cigar_length
                introns.append((intron_start, intron_end))
            if cigar_type in [0, 2, 3, 7, 8]:  # Operations that consume reference positions
                current_position += cigar_length

        # Get the strand information from the SAM flag
        if read.is_reverse:
            strand = '-'
        else:
            strand = '+'

        # Write intron coordinates with strand to the output file
        for intron_start, intron_end in introns:
            output_file.write(f"{read.reference_name}\t{intron_start}\t{intron_end}\t.\t.\t{strand}\n")

    # Close the files
    samfile.close()
    output_file.close()

In [3]:
# Extracting the introns from all the BAM files and outputting them as a .BED file
get_introns_from_bam(infile="aln_a_sorted.bam", outfile="./data/false_introns_simulated/simulation_a_introns.bed")
get_introns_from_bam(infile="aln_b_sorted.bam", outfile="./data/false_introns_simulated/simulation_b_introns.bed")
get_introns_from_bam(infile="aln_c_sorted.bam", outfile="./data/false_introns_simulated/simulation_c_introns.bed")
get_introns_from_bam(infile="aln_d_sorted.bam", outfile="./data/false_introns_simulated/simulation_d_introns.bed")

# Moving previous files

All previous files (except final .BED files generated in previous cell) is then moved from the root directory to /data/false_introns_simulated/preprocessing

In [ ]:
df_simulated_introns1 = pd.read_csv("./data/false_introns_simulated/simulation_a_introns.bed", sep="\t", names=["chr", "start", "end", "features", "score", "strand"])
df_simulated_introns1 = df_simulated_introns1.loc[:, ["chr", "start", "end", "strand"]]
df_simulated_introns1["class"] = 0
df_simulated_introns1 = df_simulated_introns1.drop_duplicates(subset=["chr", "start", "end", "strand"])
df_simulated_introns1

In [ ]:
df_simulated_introns2 = pd.read_csv("./data/false_introns_simulated/introns2.bed", sep="\t", names=["chr", "start", "end", "features", "score", "strand"])
df_simulated_introns2 = df_simulated_introns2.loc[:, ["chr", "start", "end", "strand"]]
df_simulated_introns2["class"] = 0
df_simulated_introns2 = df_simulated_introns2.drop_duplicates(subset=["chr", "start", "end", "strand"])
df_simulated_introns2

In [ ]:
df_simulated_introns3 = pd.read_csv("./data/false_introns_simulated/introns_pacbio_1.bed", sep="\t", names=["chr", "start", "end", "features", "score", "strand"])
df_simulated_introns3 = df_simulated_introns3.loc[:, ["chr", "start", "end", "strand"]]
df_simulated_introns3["class"] = 0
df_simulated_introns3 = df_simulated_introns3.drop_duplicates(subset=["chr", "start", "end", "strand"])
df_simulated_introns3

In [ ]:
df_simulated_introns4 = pd.read_csv("./data/false_introns_simulated/introns_pacbio_2.bed", sep="\t", names=["chr", "start", "end", "features", "score", "strand"])
df_simulated_introns4 = df_simulated_introns3.loc[:, ["chr", "start", "end", "strand"]]
df_simulated_introns4["class"] = 0
df_simulated_introns4 = df_simulated_introns3.drop_duplicates(subset=["chr", "start", "end", "strand"])
df_simulated_introns4

In [ ]:
df_simulated_introns5 = pd.read_csv("./data/false_introns_simulated/introns_ont_lncRNA.bed", sep="\t", names=["chr", "start", "end", "features", "score", "strand"])
df_simulated_introns5 = df_simulated_introns5.loc[:, ["chr", "start", "end", "strand"]]
df_simulated_introns5["class"] = 0
df_simulated_introns5 = df_simulated_introns5.drop_duplicates(subset=["chr", "start", "end", "strand"])
df_simulated_introns5

In [ ]:
sum( pd.concat([df_merged, df_simulated_introns2, df_simulated_introns3, df_simulated_introns4]).drop_duplicates(subset=["chr", "start", "end", "strand"], keep='first') ["class"] == 0)

In [ ]:
merged_df = pd.concat([df_merged, df_simulated_introns2, df_simulated_introns3, df_simulated_introns4, df_simulated_introns5]).drop_duplicates(subset=["chr", "start", "end", "strand"], keep='first')
sum(merged_df["class"] == 0)

In [ ]:
assert(len(merged_df[merged_df.start == 15038]) == 1)
assert(merged_df[merged_df.start == 15038].iloc[0]["class"] == 1)

 Now we have 125,464 false introns!

In [ ]:
merged_df.to_csv("data/2.1-merged_train_set.tsv", sep="\t", index=False)
merged_df